# Final Project Submission
---

## Info

* Student name: **Barto Molina**
* Student pace: **part time**
* Scheduled project review date/time: **10/15/2019 5:00 PM (EST)**
* Instructor name: **Victor Geislinger**
* Blog post URL: [...](https://medium.com/@bartomolina/...)

## The Project

The goal of the project is to find the best 5 zipcodes across the US to invest in and model and project the median price for those zipcodes in the coming years. We're going to select these top zipcodes based on the distance to our workplace and the ROI from the past years.

We obtained the data from the [Zillow Research website](https://www.zillow.com/research/data/), which uses the ZHVI (Zillow Home Value Index) - A smoothed, seasonally adjusted measure of the median estimated home value across a given region and housing type. We've used the ZHVI for All Homes (SFR, Condo/Co-op) dataset.

## Imports

We'll import the required libraries that will be used throughout the rest of the project:

In [335]:
import pandas as pd
import numpy as np
import datetime
import googlemaps
import folium
import json
import random

In [336]:
# we use dotenv to use Google API key
# use .env.example as a reference
import os
from dotenv import load_dotenv
load_dotenv()

GOOGLE_KEY = os.getenv("GOOGLE_KEY")
gmaps = googlemaps.Client(key=GOOGLE_KEY)

## 0. Parameters
---

We will use this parameters to perform different calculations:

- **arrival_time**: Date used for the distance calculations when using the Google API.
- **target_location**: Destination used to calculate the distance from the different zipcodes when using the Google API.
- **max_commuting_time**: Maximum commuting time in minutes used to calculate the top 5 zipcodes.

In [478]:
# Monday Nov. 4th. 9 AM
arrival_time = datetime.datetime(2019, 11, 4, 9, 0)
target_location = "United Nations Secretariat Building, East 42nd Street, New York, NY"
max_commuting_time = 45

## 1. Load the dataset / Data cleansing
---

In [426]:
df = pd.read_csv('zillow_data.csv', dtype={'RegionName' : str })
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,167200.0,167400.0,167400.0,...,1051500,1029200,1014300,1008700,991500,973400,965800,967100,967800,965400
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,148200.0,148900.0,149400.0,...,314500,314700,314600,312900,311000,309700,308900,308400,307000,305700
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,350900.0,351700.0,352300.0,...,1375700,1370100,1369000,1362500,1358300,1369700,1394900,1403200,1405400,1412700
3,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,4,160800.0,162700.0,164200.0,...,367100,365900,364800,362800,360300,357600,354200,348900,343800,341500
4,61616,10002,New York,NY,New York-Newark-Jersey City,New York County,5,NaN,NaN,NaN,...,966100,954500,949600,943700,928400,913300,901400,892700,891200,889900


In [430]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4770 entries, 0 to 4769
Columns: 289 entries, RegionID to 2019-09
dtypes: float64(237), int64(47), object(5)
memory usage: 10.5+ MB


As our zipcode selection is going to be based on the distance to Midtown Manhattan, I'm going to limit the results to the states of **New York**, **New Jersey** and **Conneticut**:

In [435]:
# create a new dataframe with the house price information for the states of NY, NJ and CT
df_NY = df.loc[df['State'].isin(['NY', 'NJ', 'CT'])].copy()
df_NY.reset_index(drop=True, inplace=True)

In [436]:
df_NY.shape

(466, 289)

Looking at the first few rows, We've noticed some null values. As our dataset is large enough, We can probably remove the columns / rows that contain null values. First, I'm going to check which columns contain null values and how many:

In [451]:
for col in df_NY:
    if df_NY[col].isnull().sum():
        print(f"{col} - {df_NY[col].isnull().sum()}")

Metro - 1
1996-04 - 42
1996-05 - 42
1996-06 - 42
1996-07 - 42
1996-08 - 42
1996-09 - 42
1996-10 - 42
1996-11 - 42
1996-12 - 42
1997-01 - 42
1997-02 - 42
1997-03 - 42
1997-04 - 42
1997-05 - 42
1997-06 - 42
1997-07 - 42
1997-08 - 42
1997-09 - 42
1997-10 - 42
1997-11 - 42
1997-12 - 42
1998-01 - 42
1998-02 - 42
1998-03 - 42
1998-04 - 42
1998-05 - 42
1998-06 - 42
1998-07 - 42
1998-08 - 42
1998-09 - 42
1998-10 - 42
1998-11 - 42
1998-12 - 42
1999-01 - 42
1999-02 - 42
1999-03 - 42
1999-04 - 42
1999-05 - 42
1999-06 - 42
1999-07 - 42
1999-08 - 42
1999-09 - 42
1999-10 - 42
1999-11 - 42
1999-12 - 42
2000-01 - 42
2000-02 - 42
2000-03 - 42
2000-04 - 42
2000-05 - 42
2000-06 - 42
2000-07 - 42
2000-08 - 42
2000-09 - 42
2000-10 - 42
2000-11 - 42
2000-12 - 42
2001-01 - 42
2001-02 - 42
2001-03 - 42
2001-04 - 42
2001-05 - 42
2001-06 - 42
2001-07 - 42
2001-08 - 42
2001-09 - 42
2001-10 - 42
2001-11 - 42
2001-12 - 42
2002-01 - 42
2002-02 - 42
2002-03 - 42
2002-04 - 42
2002-05 - 42
2002-06 - 42
2002-07 - 42
20

There's missing information for the first few years for almost 10% of the zipcodes. As we have data for enough years, I'm going to keep only the information for the past **10 years**:

In [452]:
df_NY.drop(df_NY.iloc[:, 7:168], axis=1, inplace=True) 

I will delete the rest of the zipcodes that contain null values for any year:

In [456]:
df_NY.dropna(inplace=True)

Now that we have removed all the null values from our dataset, let's look at the different counties:

In [459]:
df_NY['CountyName'].value_counts().head(10)

New York County     39
Bergen County       33
Queens County       30
Kings County        30
Monmouth County     28
Middlesex County    23
Nassau County       19
Morris County       17
New Haven County    16
Suffolk County      15
Name: CountyName, dtype: int64

## 2. Selecting the top 5 zipcodes
---

In order to select the top 5 zipcodes, we're going to calculate the distance to the workplace and the ROI for the last 10 years:

### First, let's calculate the distance from each zipcode to the target location

We'll use the [Google API](https://github.com/googlemaps/google-maps-services-python) to calculate the distance from each zipcode to the target location. In order to calculate the distance, we'll look at the walking and the transit distance and we'll select the fastest route. Then, we'll store the distance in minutes in a new column **Distance**:

In [464]:
df_NY['Distance'] = np.nan

for i, zipcode in df_NY.iterrows():
    directions_transit = gmaps.directions(zipcode['RegionName'], target_location, mode='transit', arrival_time=arrival_time)
    directions_walking = gmaps.directions(zipcode['RegionName'], target_location, mode='walking', arrival_time=arrival_time)
    if directions_transit or directions_walking:
        distance_transit = np.NaN if not directions_transit else directions_transit[0]['legs'][0]['duration']['value']
        distance_walking = np.NaN if not directions_walking else directions_walking[0]['legs'][0]['duration']['value']
        df_NY.at[i, 'Distance'] = np.nanmin([distance_walking, distance_transit]) / 60

We'll remove the zipcodes for which we couldn't calculate the route (if any) and limit the results to the maximum commuting time defined in the parameters:

In [501]:
df_NY.dropna(subset=['Distance'], inplace=True)
df_NY_by_dist = df_NY.loc[df_NY['Distance'] <= max_commuting_time].copy()
print(len(df_NY))
print(len(df_NY_by_dist))

439
53


Now, we'll generate a json with the geometries for each of the zipcodes contained in our result dataset and plot a map with the distance of each of the zipcodes to the workspace:

In [480]:
ny_geo_path = 'ny_new_york_zip_codes_geo.min.json'
nj_geo_path = 'nj_new_jersey_zip_codes_geo.min.json'
ct_geo_path = 'ct_connecticut_zip_codes_geo.min.json'
with open(ny_geo_path) as ny_geo_file, open(nj_geo_path) as nj_geo_file, open(ct_geo_path) as ct_geo_file:
    NY_zip = json.load(ny_geo_file)
    NJ_zip = json.load(nj_geo_file)
    CT_zip = json.load(ct_geo_file)
    
all_features = NY_zip['features'] + NJ_zip['features'] + CT_zip['features']

tristate_data = {}
tristate_data['type'] = 'FeatureCollection'
tristate_data['features'] = list()

for zipcode in all_features:
    code = zipcode['properties']['ZCTA5CE10']
    if code in df_NY_by_dist['RegionName'].values:
        zipcode['properties']['ZCTA5CE10'] = code
        distance = df_NY_by_dist.loc[df_NY_by_dist['RegionName'] == code, 'Distance'].values[0].round(2)
        zipcode['properties']['CustLabel'] = f'{code} - {distance}m'
        tristate_data['features'].append(zipcode)

tristate_json_data = json.dumps(tristate_data)

In [527]:
m = folium.Map(location=[40.75, -73.97], zoom_start=12)

choropleth = folium.Choropleth(highlight=True,
    geo_data=tristate_json_data,
    name='choropleth',
    data=df_NY_by_dist,
    columns=['RegionName', 'Distance'],
    key_on='feature.properties.ZCTA5CE10',
    fill_color='OrRd',               
    fill_opacity=0.7,
    line_opacity=0.4,
    legend_name='house prices (median)'
).add_to(m)

folium.LayerControl().add_to(m)

# add tooltip to see the zipcode
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['CustLabel'], labels=False)
)

m

There is a total of **53** zipcodes within the range of **45** minutes from work.

### Calculate the 10 yr. ROI for each of the zipcodes

We're going to calculate the 10 yr. ROI for each of the zipcodes by creating a new column in our dataframe:

In [502]:
df_NY_by_dist['ROI'] = ((df_NY_by_dist['2019-09'] - df_NY_by_dist['2009-09']) / df_NY_by_dist['2009-09'] * 100).round()

In [524]:
df_NY_by_dist.head()
df_NY.loc[df_NY['RegionName'] == '10010']

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2009-09,2009-10,2009-11,...,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,Distance,ROI
85,61624,10010,New York,NY,New York-Newark-Jersey City,New York County,631,1134500.0,1113300.0,1098400.0,...,1692200,1689600,1667000,1642000,1628600,1623100,1617900,1612900,13.683333,42.0


In [521]:
for feature in tristate_data['features']:
    code = feature['properties']['ZCTA5CE10']
    ROI = int(df_NY_by_dist.loc[df_NY_by_dist['RegionName'] == code, 'ROI'].values[0])
    feature['properties']['ROI'] = f"{ROI}%"
    
tristate_json_data = json.dumps(tristate_data)

In [526]:
m = folium.Map(location=[40.75, -73.97], zoom_start=12)

choropleth = folium.Choropleth(highlight=True,
    geo_data=tristate_json_data,
    name='choropleth',
    data=df_NY_by_dist,
    columns=['RegionName', 'ROI'],
    key_on='feature.properties.ZCTA5CE10',
    fill_color='OrRd',               
    fill_opacity=0.7,
    line_opacity=0.4,
    legend_name='ROI'
).add_to(m)

folium.LayerControl().add_to(m)

# add tooltip to see the zipcode
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['ROI'], labels=False)
)

m

### Calculate an index with the distance and the ROI

Based on the previous analysis. We're going assign each zipcode a DRI (Distance - ROI index). We're going to weight both parameters and calculate this index as follows:

In [531]:
df_NY_by_dist['DRI'] = (1 / df_NY_by_dist['Distance']) + (df_NY_by_dist['ROI'] / 100)

In [554]:
df_NY_Top_5 = df_NY_by_dist.sort_values(by='DRI', ascending=False).head().copy()
df_NY_Top_5

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2009-09,2009-10,2009-11,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,Distance,ROI,DRI
12,62017,11206,New York,NY,New York-Newark-Jersey City,Kings County,26,384100.0,381800.0,382500.0,...,742600,740600,739500,740300,740600,742500,742600,37.233333,93.0,0.956858
94,62033,11222,New York,NY,New York-Newark-Jersey City,Kings County,748,497200.0,500100.0,504700.0,...,951100,952200,956400,959200,956700,951900,947200,35.950000,91.0,0.937816
53,62004,11101,New York,NY,New York-Newark-Jersey City,Queens County,270,597100.0,592000.0,588100.0,...,1119900,1129100,1130600,1129700,1128100,1122800,1118200,26.100000,87.0,0.908314
118,62058,11249,New York,NY,New York-Newark-Jersey City,Kings County,1121,602600.0,594300.0,588900.0,...,1085700,1077100,1049000,1047300,1097000,1142000,1129100,40.466667,87.0,0.894712
27,62022,11211,New York,NY,New York-Newark-Jersey City,Kings County,75,531000.0,527200.0,525700.0,...,958800,959600,961300,964000,968300,970100,967100,40.833333,82.0,0.844490


In [561]:
top5_data = {}
top5_data['type'] = 'FeatureCollection'
top5_data['features'] = list()

for zipcode in all_features:
    code = zipcode['properties']['ZCTA5CE10']
    if code in df_NY_Top_5['RegionName'].values:
        zipcode['properties']['ZCTA5CE10'] = code
        distance = df_NY_Top_5.loc[df_NY_by_dist['RegionName'] == code, 'Distance'].values[0].round(2)
        ROI = int(df_NY_Top_5.loc[df_NY_Top_5['RegionName'] == code, 'ROI'].values[0])
        zipcode['properties']['LabelTop5'] = f"{code} - {distance}m - {ROI}%"
        top5_data['features'].append(zipcode)

top5_json_data = json.dumps(top5_data)

In [562]:
m = folium.Map(location=[40.75, -73.97], zoom_start=12)

choropleth = folium.Choropleth(highlight=True,
    geo_data=top5_json_data,
    name='choropleth',
    data=df_NY_Top_5,
    columns=['RegionName', 'ROI'],
    key_on='feature.properties.ZCTA5CE10',
    fill_color='OrRd',               
    fill_opacity=0.7,
    line_opacity=0.4,
    legend_name='Top 5 zipcodes'
).add_to(m)

folium.LayerControl().add_to(m)

# add tooltip to see the zipcode
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['LabelTop5'], labels=False)
)

m

So, given our calculations, the best zipcodes to buy a property in NY would be:

- **Brooklyn**:
  - 11206: 37m - 93%
  - 11222: 36m - 91%
  - 11211: 41m - 82%
  - 11249


- **Astoria**:
  - 11101: 26m - 87%

In [ ]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

In [ ]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [ ]:
df_NY.drop(['RegionID', 'SizeRank'], axis=1, inplace=True)

In [ ]:
melt_data(df_NY)

## Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

## Step 5: ARIMA Modeling

## Step 6: Interpreting Results

## Future work

- Better calculate distance (currently just an average point in the zipcode)
- Better function weighting roi vs. distance